In [1]:
import numpy as np
import pandas as pd
import time
import lightkurve as lk
from lightkurve import TessLightCurve, LightCurveFile
import matplotlib.pyplot as plt
from astropy.io import fits
import astropy
%matplotlib inline

import triceratops.triceratops as tr

In [78]:
def get_aperture(pixels):
    aperture_list = []
    current_row = pixels.row
    for row in pixels.pipeline_mask:
        current_column = pixels.column
        for cell in row:
            if cell == True:
                aperture_list.append([current_column, current_row])
            current_column = current_column + 1
        current_row = current_row + 1
    return aperture_list

In [131]:
def get_triceratops_planet_candidate_probability(x):
    kic = x[1]
    ID = x[3] # a.k.a. TIC
    print(ID)
    disposition = x[4]
    planet_number = x[6]
    period = x[8]
    depth_ppm = x[10]
    depth = depth_ppm / 1000000
    
    # get target pixel file
    try:
        pixels = lk.search_targetpixelfile("TIC " + str(ID), mission="TESS").download()
        
        # get sector and aperture
        sectors = [pixels.sector]
        ap = np.array(get_aperture(pixels))
        apertures = np.array([ap])

        # create triceratops object for star
        target = tr.target(ID=ID, sectors=sectors)
        target.calc_depths(tdepth=depth, all_ap_pixels=apertures)

        # get lightcurve data and process it
        lightcurve = pixels.to_lightcurve()
        exoplanet = lightcurve.flatten().remove_outliers().fold(period=period)

        # generate and save plot
        plt.plot(exoplanet.time, exoplanet.flux)
        plt.xlabel("Time")
        plt.ylabel("Flux")
        if disposition == "FP":
            status = "FALSE POSITIVE"
        else:
            status = "CANDIDATE"
        plot_title = str(kic) + "-" + str(planet_number) + " (" + status + ")"
        plt.title(plot_title)
        plot_filename = str(kic) + "_" + str(planet_number) + ".png"
        plt.savefig(plot_filename)
        plt.close()

        # bin the data a little so it runs faster
        bins = np.linspace(exoplanet.time.min(),exoplanet.time.max(),11)
        stdvs = np.zeros(10)
        for j in range(bins.shape[0]-1):
            this_stdv = np.std(exoplanet.flux[(exoplanet.time > bins[j]) & (exoplanet.time < bins[j+1])])
            stdvs[j] = this_stdv
        sigma = np.mean(stdvs)
        lc_bins = int(exoplanet.time.shape[0]/100)
        lc = TessLightCurve(exoplanet.time, exoplanet.flux).bin(lc_bins, method='mean')

        lc = lc.remove_nans()

        # calculate the probability of a planet candidate
        target.calc_probs(time=lc.time, flux_0=lc.flux, sigma_0=sigma, P_orb=period)

        return np.round(1-target.FPP, 4)
    
    except Exception as e:
        return -1

In [132]:
df = pd.read_csv("joined_with_tess_tp_counts_test.csv")
df.head()

,Unnamed: 0,kic,TCE_ID,tic,disposition,not_transit,pn,n_plan,period,target_pixel_files,depth
0,11983,11403032,011403032-01,26541137,FP,1,1,1,7.631403,1,32.060572
1,12025,11453915,011453915-01,350810416,FP,1,1,1,24.099985,1,44.280683
2,12061,11494130,011494130-01,48418928,FP,1,1,1,18.955066,1,909.032692
3,12095,11512820,011512820-01,27186754,FP,1,1,1,38.926088,1,39.465696
4,12164,11559725,011559725-01,424815274,FP,1,1,1,424.922547,1,80.870525


In [133]:
df.loc[df["target_pixel_files"] >= 1, "triceratops_probability"] = df[df["target_pixel_files"] >= 1].apply(get_triceratops_planet_candidate_probability, axis=1)

26541137


/Users/deanwang/anaconda3/envs/triceratops/lib/python3.8/site-packages/lightkurve/search.py:228: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn('Warning: {} files available to download. '


TRILEGAL form submitted.
Calculating TP, EB, and EBx2P scenario probabilities for 26541137.
Calculating PTP, PEB, and PEBx2P scenario probabilities for 26541137.
Calculating STP, SEB, and SEBx2P scenario probabilities for 26541137.
Calculating DTP, DEB, and DEBx2P scenario probabilities for 26541137.
Calculating BTP, BEB, and BEBx2P scenario probabilities for 26541137.
Calculating NTP, NEB, and NEBx2P scenario probabilities for 1882947785.
Calculating NTP, NEB, and NEBx2P scenario probabilities for 1882947782.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882947807.
Calculating NTP, NEB, and NEBx2P scenario probabilities for 1882947776.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882947779.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882947808.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882947803.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 26541141.
Calculating NTP, NEB, and NEB2xP scenario probabilitie

/Users/deanwang/anaconda3/envs/triceratops/lib/python3.8/site-packages/lightkurve/search.py:228: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn('Warning: {} files available to download. '


TRILEGAL form submitted.
Calculating TP, EB, and EBx2P scenario probabilities for 350810416.
Calculating PTP, PEB, and PEBx2P scenario probabilities for 350810416.
Calculating STP, SEB, and SEBx2P scenario probabilities for 350810416.
Calculating DTP, DEB, and DEBx2P scenario probabilities for 350810416.
Calculating BTP, BEB, and BEBx2P scenario probabilities for 350810416.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 350810415.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882971566.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 350810419.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 350810413.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882971577.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 350810429.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 350810432.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882971551.
Calculating NTP, NEB, and NEB2xP scenario probabilit

...

TRILEGAL form submitted.
Calculating TP, EB, and EBx2P scenario probabilities for 28227113.
Calculating PTP, PEB, and PEBx2P scenario probabilities for 28227113.
Calculating STP, SEB, and SEBx2P scenario probabilities for 28227113.
Calculating DTP, DEB, and DEBx2P scenario probabilities for 28227113.
Calculating BTP, BEB, and BEBx2P scenario probabilities for 28227113.
Calculating NTP, NEB, and NEBx2P scenario probabilities for 1882065270.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 28227110.
Calculating NTP, NEB, and NEBx2P scenario probabilities for 1882065273.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 28227107.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 28227101.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882065276.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1882065275.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 28227109.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 

/Users/deanwang/anaconda3/envs/triceratops/lib/python3.8/site-packages/lightkurve/search.py:228: LightkurveWarning: Warning: 3 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn('Warning: {} files available to download. '


TRILEGAL form submitted.
Calculating TP, EB, and EBx2P scenario probabilities for 267666834.
Calculating PTP, PEB, and PEBx2P scenario probabilities for 267666834.
Calculating STP, SEB, and SEBx2P scenario probabilities for 267666834.
Calculating DTP, DEB, and DEBx2P scenario probabilities for 267666834.
Calculating BTP, BEB, and BEBx2P scenario probabilities for 267666834.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 267666833.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 48448189.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 267666825.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 267666826.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 48448193.
Calculating NTP, NEB, and NEBx2P scenario probabilities for 267666841.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1717224238.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 267666844.
27083004


/Users/deanwang/anaconda3/envs/triceratops/lib/python3.8/site-packages/lightkurve/search.py:228: LightkurveWarning: Warning: 2 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn('Warning: {} files available to download. '


TRILEGAL form submitted.
Calculating TP, EB, and EBx2P scenario probabilities for 27083004.
Calculating PTP, PEB, and PEBx2P scenario probabilities for 27083004.
Calculating STP, SEB, and SEBx2P scenario probabilities for 27083004.
Calculating DTP, DEB, and DEBx2P scenario probabilities for 27083004.
Calculating BTP, BEB, and BEBx2P scenario probabilities for 27083004.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 27082997.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1883070288.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1883070339.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 27083008.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 1883070283.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 27083020.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 27082990.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 27083003.
Calculating NTP, NEB, and NEB2xP scenario probabilities for 27

In [134]:
df.head()

,Unnamed: 0,kic,TCE_ID,tic,disposition,not_transit,pn,n_plan,period,target_pixel_files,depth,triceratops_probability
0,11983,11403032,011403032-01,26541137,FP,1,1,1,7.631403,1,32.060572,0.8309
1,12025,11453915,011453915-01,350810416,FP,1,1,1,24.099985,1,44.280683,0.5902
2,12061,11494130,011494130-01,48418928,FP,1,1,1,18.955066,1,909.032692,0.7513
3,12095,11512820,011512820-01,27186754,FP,1,1,1,38.926088,1,39.465696,NaN
4,12164,11559725,011559725-01,424815274,FP,1,1,1,424.922547,1,80.870525,0.5036
